<a href="https://colab.research.google.com/github/Takuchan/RobocupImage/blob/tf_takumi/%E5%9F%BA%E6%9C%AC%E7%9A%84%E3%81%AA%E3%83%86%E3%82%AD%E3%82%B9%E3%83%88%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 感情レビューのテキスト分類

In [11]:
import matplotlib.pyplot as plot
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses

In [12]:
print(tf.__version__)

2.14.0


### メモ
映画レビューのテキストを使用して、それが肯定的であるか否定的であるかに分類するように感情分析モデルをトレーニングします。これは二項分類の例で機械学習問題では重要な分類法として広く適用される。

In [13]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

### メモ
すべてのデータを表示。
Ubuntuコマンドでいう`$ ls`にあたる。

In [14]:
os.listdir(dataset_dir)

['README', 'imdb.vocab', 'test', 'train', 'imdbEr.txt']

In [15]:
train_dir = os.path.join(dataset_dir,'train') # ２つの文字列を一つにしたパスを生成できる
os.listdir(train_dir)


['urls_pos.txt',
 'neg',
 'urls_unsup.txt',
 'unsupBow.feat',
 'unsup',
 'urls_neg.txt',
 'pos',
 'labeledBow.feat']

In [16]:
sample_file = os.path.join(train_dir, 'pos/1181_9.txt')
with open(sample_file) as f:
  print(f.read())

Rachel Griffiths writes and directs this award winning short film. A heartwarming story about coping with grief and cherishing the memory of those we've loved and lost. Although, only 15 minutes long, Griffiths manages to capture so much emotion and truth onto film in the short space of time. Bud Tingwell gives a touching performance as Will, a widower struggling to cope with his wife's death. Will is confronted by the harsh reality of loneliness and helplessness as he proceeds to take care of Ruth's pet cow, Tulip. The film displays the grief and responsibility one feels for those they have loved and lost. Good cinematography, great direction, and superbly acted. It will bring tears to all those who have lost a loved one, and survived.


In [17]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir) # フォルダーの中身ごと削除するコマンド

In [18]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


### メモ
上記のようにトレーニングフォルダーには25,000の例があり、そのうち、80%をトレーニングに使用する。


In [36]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(3):
    print("Review", text_batch.numpy()[i])
    print("Label", label_batch.numpy()[i])

Review b"I went to see Fever Pitch with my Mom, and I can say that we both loved it. It wasn't the typical romantic comedy where someone is pining for the other, and blah blah blah... You weren't waiting for the climatic first kiss or for them to finally get together. It was more real, because you saw them through the relationship, rather than the whole movie be about them getting together. People could actually relate to the film, because it didn't seem like extraordinary circumstances, or impossible situations. It was really funny, and I think it was Jimmy Fallon's best performance. All in all... I would definitely recommend it!"
Label 1
Review b"from the view of a NASCAR Maniac like I am, the movie is interesting. You can see many race cars from 1983. Even tough, the racing scenes are not that much realistic. But I have to admit, that I haven't seen any race before 1995, because before that time, they didn't show any NASCAR races in Germany)<br /><br />from the view of a Burt Reynol

In [20]:
raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [21]:
raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/test',
    batch_size=batch_size)

Found 25000 files belonging to 2 classes.


### メモ
トレーニング洋データを準備する。
`tf.keras.layers.TextVectorization`レイヤーを使用して、データを標準化、トークン化、ベクトル化します。
標準化とは、テキストを前処理することを指す。
- 標準化句読点やHTML要素を削除してデータセットを簡素化する。
- トークン化→文字列をトークンに分割すること（例えば空白で分割することにより、文を個々の単語に分割できる）
- ベクトル化→トークンを数値に変換してニューラルネットワークに入力できるようにすること



In [22]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase,'<br />', ' ')
  return tf.strings.regex_replace(stripped_html,'[%s]' % re.escape(string.punctuation),'')


### メモ
次に`TextVectorization`レイヤを作成します。このレイヤを使用して
データを標準化、トークン化、ベクトル化する。output_modeをintに設定して、トークンごとに一意の整数インデックスを作成します。
デフォルトの分割関数と上記で定義したカスタム表ズン化関数を仕様していることに注意してください。また明示的な最大値`sequence_length`などモデルの定数をいくつか定義する。

In [23]:
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize = custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length
)

In [24]:
train_text = raw_train_ds.map(lambda x, y:x)
vectorize_layer.adapt(train_text)

In [25]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [26]:
# retrieve a batch (of 32 reviews and labels) from the dataset
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))

Review tf.Tensor(b'Great movie - especially the music - Etta James - "At Last". This speaks volumes when you have finally found that special someone.', shape=(), dtype=string)
Label neg
Vectorized review (<tf.Tensor: shape=(1, 250), dtype=int64, numpy=
array([[  86,   17,  260,    2,  222,    1,  571,   31,  229,   11, 2418,
           1,   51,   22,   25,  404,  251,   12,  306,  282,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
       

In [27]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [28]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [29]:
embedding_dim = 16

In [30]:
model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          160016    
                                                                 
 dropout (Dropout)           (None, None, 16)          0         
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 160033 (625.13 KB)
Trainable params: 160033 (625.13 KB)
Non-trainable params: 0 (0.00 Byte)
________________

In [31]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [32]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/10
625/625 [==============================] - 61s 88ms/step - loss: 0.6636 - binary_accuracy: 0.6970 - val_loss: 0.6137 - val_binary_accuracy: 0.7726
Epoch 2/10
625/625 [==============================] - 3s 4ms/step - loss: 0.5468 - binary_accuracy: 0.8027 - val_loss: 0.4967 - val_binary_accuracy: 0.8224
Epoch 3/10
625/625 [==============================] - 3s 4ms/step - loss: 0.4431 - binary_accuracy: 0.8459 - val_loss: 0.4190 - val_binary_accuracy: 0.8486
Epoch 4/10
625/625 [==============================] - 3s 4ms/step - loss: 0.3771 - binary_accuracy: 0.8676 - val_loss: 0.3727 - val_binary_accuracy: 0.8614
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 0.3343 - binary_accuracy: 0.8801 - val_loss: 0.3445 - val_binary_accuracy: 0.8674
Epoch 6/10
625/625 [==============================] - 4s 6ms/step - loss: 0.3049 - binary_accuracy: 0.8886 - val_loss: 0.3255 - val_binary_accuracy: 0.8716
Epoch 7/10
625/625 [==============================] - 3s 4ms/s

### 最後に
学習したデータをエクスポートする


In [33]:
export_model = tf.keras.Sequential([
  vectorize_layer,
  model,
  layers.Activation('sigmoid')
])

export_model.compile(
    loss=losses.BinaryCrossentropy(from_logits=False), optimizer="adam", metrics=['accuracy']
)

# Test it with `raw_test_ds`, which yields raw strings
loss, accuracy = export_model.evaluate(raw_test_ds)
print(accuracy)

782/782 [==============================] - 6s 6ms/step - loss: 0.3102 - accuracy: 0.8732
0.8732399940490723


In [34]:
examples = [
    "love",
]

export_model.predict(examples)


1/1 [==============================] - 0s 141ms/step


array([[0.5515418]], dtype=float32)

In [35]:
!ls

aclImdb  aclImdb_v1.tar.gz  sample_data
